# 第4章 多层感知机

## 4.1 多层感知机

### 练习4.1.1

计算pReLU激活函数的导数。

**解答：**

### 练习4.1.2

证明一个仅使用ReLU（或pReLU）的多层感知机构造了一个连续的分段线性函数。

**解答：**

### 练习4.1.3

证明$\operatorname{tanh}(x) + 1 = 2 \operatorname{sigmoid}(2x)$。

**解答：**

### 练习4.1.4

假设我们有一个非线性单元，将它一次应用于一个小批量的数据。这会导致什么样的问题？

**解答：**

## 4.2 多层感知机的从零开始实现

### 练习4.2.1

在所有其他参数保持不变的情况下，更改超参数`num_hiddens`的值，并查看此超参数的变化对结果有何影响。确定此超参数的最佳值。

**解答：**

### 练习4.2.2

尝试添加更多的隐藏层，并查看它对结果有何影响。

**解答：**

### 练习4.2.3

改变学习速率会如何影响结果？保持模型架构和其他超参数（包括轮数）不变，学习率设置为多少会带来最好的结果？

**解答：**

### 练习4.2.4

通过对所有超参数（学习率、轮数、隐藏层数、每层的隐藏单元数）进行联合优化，可以得到的最佳结果是什么？

**解答：**

### 练习4.2.5

描述为什么涉及多个超参数更具挑战性。

**解答：**

### 练习4.2.6

如果想要构建多个超参数的搜索方法，请想出一个聪明的策略。

**解答：**

## 4.3 多层感知机的简洁实现

### 练习4.3.1

尝试添加不同数量的隐藏层（也可以修改学习率），怎么样设置效果最好？

**解答：**

### 练习4.3.2

尝试不同的激活函数，哪个效果最好？

**解答：**

### 练习4.3.3

尝试不同的方案来初始化权重，什么方法效果最好？

**解答：**

## 4.4 模型选择、欠拟合和过拟合

### 练习4.4.1

这个多项式回归问题可以准确地解出吗？提示：使用线性代数。

**解答：**

### 练习4.4.2

考虑多项式的模型选择。
1. 绘制训练损失与模型复杂度（多项式的阶数）的关系图。观察到了什么？需要多少阶的多项式才能将训练损失减少到0?
2. 在这种情况下绘制测试的损失图。
3. 生成同样的图，作为数据量的函数。

**解答：**

### 练习4.4.3

如果不对多项式特征$x^i$进行标准化($1/i!$)，会发生什么事情？能用其他方法解决这个问题吗？

**解答：**

### 练习4.4.4

泛化误差可能为零吗？

**解答：**

## 4.5 权重衰减

### 练习4.5.1

在本节的估计问题中使用$\lambda$的值进行实验。绘制训练和测试精度关于$\lambda$的函数。观察到了什么？

**解答：**

### 练习4.5.2

使用验证集来找到最佳值$\lambda$。它真的是最优值吗？这有关系吗？

**解答：**

### 练习4.5.3

如果我们使用$\sum_i |w_i|$作为我们选择的惩罚（$L_1$正则化），那么更新方程会是什么样子？

**解答：**

### 练习4.5.4

我们知道$\|\mathbf{w}\|^2 = \mathbf{w}^\top \mathbf{w}$。能找到类似的矩阵方程吗（见 :numref:`subsec_lin-algebra-norms` 中的

**解答：**

### 练习4.5.5

回顾训练误差和泛化误差之间的关系。除了权重衰减、增加训练数据、使用适当复杂度的模型之外，还能想出其他什么方法来处理过拟合？

**解答：**

### 练习4.5.6

在贝叶斯统计中，我们使用先验和似然的乘积，通过公式$P(w \mid x) \propto P(x \mid w) P(w)$得到后验。如何得到带正则化的$P(w)$？

**解答：**

## 4.6 暂退法

### 练习4.6.1

如果更改第一层和第二层的暂退法概率，会发生什么情况？具体地说，如果交换这两个层，会发生什么情况？设计一个实验来回答这些问题，定量描述该结果，并总结定性的结论。

**解答：**

### 练习4.6.2

增加训练轮数，并将使用暂退法和不使用暂退法时获得的结果进行比较。

**解答：**

### 练习4.6.3

当应用或不应用暂退法时，每个隐藏层中激活值的方差是多少？绘制一个曲线图，以显示这两个模型的每个隐藏层中激活值的方差是如何随时间变化的。

**解答：**

### 练习4.6.4

为什么在测试时通常不使用暂退法？

**解答：**

### 练习4.6.5

以本节中的模型为例，比较使用暂退法和权重衰减的效果。如果同时使用暂退法和权重衰减，会发生什么情况？结果是累加的吗？收益是否减少（或者说更糟）？它们互相抵消了吗？

**解答：**

### 练习4.6.7

如果我们将暂退法应用到权重矩阵的各个权重，而不是激活值，会发生什么？

**解答：**

### 练习4.6.8

发明另一种用于在每一层注入随机噪声的技术，该技术不同于标准的暂退法技术。尝试开发一种在Fashion-MNIST数据集（对于固定架构）上性能优于暂退法的方法。

**解答：**

## 4.7 前向传播、反向传播和计算图

### 练习4.7.1 

假设一些标量函数$\mathbf{X}$的输入$\mathbf{X}$是$n \times m$矩阵。$f$相对于$\mathbf{X}$的梯度维数是多少？

**解答：**

### 练习4.7.2

向本节中描述的模型的隐藏层添加偏置项（不需要在正则化项中包含偏置项）。

1. 画出相应的计算图。
2. 推导正向和反向传播方程。

**解答：**

### 练习4.7.3

计算本节所描述的模型，用于训练和预测的内存占用。

**解答：**

### 练习4.7.4

假设想计算二阶导数。计算图发生了什么？预计计算需要多长时间？

**解答：**

### 练习4.7.5

假设计算图对当前拥有的GPU来说太大了。

1. 请试着把它划分到多个GPU上。
2. 与小批量训练相比，有哪些优点和缺点？

**解答：**

## 4.8 数值稳定性和模型初始化

### 练习4.8.1

除了多层感知机的排列对称性之外，还能设计出其他神经网络可能会表现出对称性且需要被打破的情况吗？

**解答：**

### 练习4.8.2

我们是否可以将线性回归或softmax回归中的所有权重参数初始化为相同的值？

**解答：**

### 练习4.8.3

在相关资料中查找两个矩阵乘积特征值的解析界。这对确保梯度条件合适有什么启示？

**解答：**

### 练习4.8.4

如果我们知道某些项是发散的，我们能在事后修正吗？看看关于按层自适应速率缩放的论文。

**解答：**

## 4.9 环境和分布偏移

### 练习4.9.1

当我们改变搜索引擎的行为时会发生什么？用户可能会做什么？广告商呢？

**解答：**

### 练习4.9.2

实现一个协变量偏移检测器。提示：构建一个分类器。

**解答：**

### 练习4.9.3

实现协变量偏移纠正。

**解答：**

### 练习4.9.4

除了分布偏移，还有什么会影响经验风险接近真实风险的程度？

**解答：**

## 4.10 实战Kaggle比赛：预测房价

### 练习4.10.1

能通过直接最小化价格的对数来改进模型吗？如果试图预测价格的对数而不是价格，会发生什么？

**解答：**

### 练习4.10.2

用平均值替换缺失值总是好主意吗？提示：能构造一个不随机丢失值的情况吗？

**解答：**

### 练习4.10.3

通过$K$折交叉验证调整超参数，从而提高Kaggle的得分。

**解答：**

### 练习4.10.4

通过改进模型（例如，层、权重衰减和dropout）来提高分数。

**解答：**

### 练习4.10.5

如果我们没有像本节所做的那样标准化连续的数值特征，会发生什么？